<div style = "text-align: left"><font size = 6 color = "#B22222" face = "verdana"><b>Helsinki Service Property Energy Analysis</b></font></div>
<div style = "text-align: left"><font size = 4 color = "#00008B" face = "verdana"><b>Exploratory data analysis</b></font></div>
<div style = "text-align: leftt"><font><i>Jerry & Mayen</i></font></div>
<div style = "text-align: leftt"><font>18th Feb 2023</font></div>

### Introduction

The API relates to energy consumption data of some Utility and  Service Properties in Helsinki

Nuuka open API provides a platform for getting information on the energy data of of Helsinki Utility and Service Properties. 
The endpoint is useful for understanding the energy consumption profile and other additional informations such as heating, 
water and cooling. Energy can be fetched on ```hourly```, ```daily``` and ```monthly``` basis.

The API requires no authentication. Precise energy data are generally available from 2015 onwards. 
The data is updated once a day, and the result is returned in JSON format. data are usually not 
validated hence may contain errors


In [52]:
import requests
import warnings
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import helper as hp

### Notebook settings 

In [53]:
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: '%.2f' % x)
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### List of properties

In [54]:
# decalre the url for the list of properties
list_url = 'https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List'

# call the function for getting the list of all properties from helper.py
property_list = hp.total_list_of_properties(list_url)

# print the total number of properties and the dataframe returned by the function 
print(f"number of properties:{property_list['propertyName'].nunique()}")
property_list.head()

number of properties:842


,locationName,propertyName,propertyCode
0,1000 Hakaniemen kauppahalli,1000 Hakaniemen kauppahalli,091-011-9902-0101
1,1001 Hietalahden kauppahalli,1001 Hietalahden kauppahalli,091-004-9902-0008
2,1002 Vanha kauppahalli,1002 Vanha kauppahalli,091-003-9906-0101
3,1037 Vuotalo,1037 Vuotalo,091-054-0179-0003
4,1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen,1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen,091-040-0024-0003


### Get hourly, daily and  monthly consumption for the year 2021 and 2022 

# 2022

### Electricity

In [55]:
hourly_elect_2022_df = hp.get_request_as_df(period='Hourly',
record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Electricity",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [56]:
print(f"dataframe shape:{hourly_elect_2022_df.shape}")
hourly_elect_2022_df.head()

dataframe shape:(8736, 5)


,timestamp,reportingGroup,locationName,value,unit
0,2022-01-01 00:00:00,Electricity,1000 Hakaniemen kauppahalli,0.04,kWh
1,2022-01-01 01:00:00,Electricity,1000 Hakaniemen kauppahalli,0.05,kWh
2,2022-01-01 02:00:00,Electricity,1000 Hakaniemen kauppahalli,0.05,kWh
3,2022-01-01 03:00:00,Electricity,1000 Hakaniemen kauppahalli,0.06,kWh
4,2022-01-01 04:00:00,Electricity,1000 Hakaniemen kauppahalli,0.04,kWh


### Water

In [57]:
hourly_water_2022_df = hp.get_request_as_df(period='Hourly',
record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Water",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [58]:
print(f"dataframe shape:{hourly_water_2022_df.shape}")
hourly_water_2022_df.head()

dataframe shape:(2, 5)


,timestamp,reportingGroup,locationName,value,unit
0,2022-09-08,Water,1000 Hakaniemen kauppahalli,7173,M3
1,2022-11-30,Water,1000 Hakaniemen kauppahalli,961,M3


### Heat

In [59]:
hourly_heat_2022_df = hp.get_request_as_df(period='Hourly',
record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Heat",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=True,
version=1.0)

In [60]:
print(f"dataframe shape:{hourly_heat_2022_df.shape}")
hourly_heat_2022_df.head()

dataframe shape:(8544, 5)


,timestamp,reportingGroup,locationName,value,unit
0,2022-01-01 00:00:00,Heat,1000 Hakaniemen kauppahalli,110.10,kWh
1,2022-01-01 01:00:00,Heat,1000 Hakaniemen kauppahalli,111.30,kWh
2,2022-01-01 02:00:00,Heat,1000 Hakaniemen kauppahalli,110.80,kWh
3,2022-01-01 03:00:00,Heat,1000 Hakaniemen kauppahalli,110.00,kWh
4,2022-01-01 04:00:00,Heat,1000 Hakaniemen kauppahalli,111.70,kWh


In [65]:
# list of location names with space replacement
location_names = [location.replace(" ", "%20") for location in property_list['locationName']]

# List to store URLs with valid JSON data
valid_urls = []

# Fetching data for each location for the 2022
for location in location_names:
    url = f"https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString={location}&ReportingGroup=Electricity&StartTime=2022-01-01&EndTime=2022-12-31"
    response = requests.get(url)
    data = response.json()
    if 'errorCode' not in data or data['errorCode'] != 'MissingSettingsException':
        valid_urls.append(url)

# Creating a folder to store the JSON files
folder_name = "energy_data_2022"
os.makedirs(folder_name, exist_ok=True)

# Save data from each valid URL as a separate JSON file in the folder
for index, url in enumerate(valid_urls):
    response = requests.get(url)
    data = response.json()
    filename = f"data_{index}.json"
    file_path = os.path.join(folder_name, filename)
    with open(file_path, 'w') as file:
        json.dump(data, file)
    print(f"Data saved for URL: {url} (filename: {filename})")

print("Data saved successfully.")


KeyboardInterrupt: 

# 2023

### District Cooling

In [61]:
daily_elect_df = hp.get_request_as_df(period='Hourly',
record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Electricity",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

## function to get hourly, daily or monthly data

In [ ]:
check = get_request_as_json(period='Hourly',
record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Water",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [ ]:
def get_daily_consumption(
    record=None,
    search_string=None,
    reporting_group=None,
    start_time=None,
    end_time=None,
    Normalization=True,
    version=None
):
    # get a dictionary of all possible arguement
    all_args = locals()
    
    #fiter out the set arguements
    set_args = {k: v for k,v in all_args.items() if v is not None and k !='Normalization'}# and k != 'Normalization'
    
    # get a list of all values from the dict "set_args"
    set_args_list = list(set_args.values())
    
    if 'PropertyCode' in list(set_args.values()):
        #construct the url for API access
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Daily/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a1,b1,c1,d1,e1,f1 = set_args_list[5],set_args_list[0],set_args_list[1],set_args_list[2],set_args_list[3],set_args_list[4]
        access_link1 = url.format(a1,b1,c1,d1,e1,f1)
        response1 = requests.get(access_link1)
        data1 = response1.text
        data_df1 = pd.read_json(data1)
        return data_df1
        
    else:
        
        #construct the url for API access
        record_1 = set_args_list[1].replace(" ","%20")
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Daily/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a2,b2,c2,d2,e2,f2 = set_args_list[5],set_args_list[0],record_1,set_args_list[2],set_args_list[3],set_args_list[4]
        access_link2 = url.format(a2,b2,c2,d2,e2,f2)
        response2 = requests.get(access_link2)
        data2 = response2.text
        data_df2 = pd.read_json(data2)
        return data_df2
    
def get_monthly_consumption(
    record=None,
    search_string=None,
    reporting_group=None,
    start_time=None,
    end_time=None,
    Normalization=True,
    version=None
):
    # get a dictionary of all possible arguement
    all_args = locals()
    
    #fiter out the set arguements
    set_args = {k: v for k,v in all_args.items() if v is not None and k !='Normalization'}# and k != 'Normalization'
    
    # get a list of all values from the dict "set_args"
    set_args_list = list(set_args.values())
    
    if 'PropertyCode' in list(set_args.values()):
        #construct the url for API access
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Monthly/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a1,b1,c1,d1,e1,f1 = set_args_list[5],set_args_list[0],set_args_list[1],set_args_list[2],set_args_list[3],set_args_list[4]
        access_link1 = url.format(a1,b1,c1,d1,e1,f1)
        response1 = requests.get(access_link1)
        data1 = response1.text
        data_df1 = pd.read_json(data1)
        return data_df1
        
    else:
        
        #construct the url for API access
        record_1 = set_args_list[1].replace(" ","%20")
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Monthly/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a2,b2,c2,d2,e2,f2 = set_args_list[5],set_args_list[0],record_1,set_args_list[2],set_args_list[3],set_args_list[4]
        access_link2 = url.format(a2,b2,c2,d2,e2,f2)
        response2 = requests.get(access_link2)
        data2 = response2.text
        data_df2 = pd.read_json(data2)
        return data_df2  
    
def get_hourly_consumption(
    record=None,
    search_string=None,
    reporting_group=None,
    start_time=None,
    end_time=None,
    Normalization=True,
    version=None
):
    # get a dictionary of all possible arguement
    all_args = locals()
    
    #fiter out the set arguements
    set_args = {k: v for k,v in all_args.items() if v is not None and k !='Normalization'}# and k != 'Normalization'
    
    # get a list of all values from the dict "set_args"
    set_args_list = list(set_args.values())
    
    if 'PropertyCode' in list(set_args.values()):
        #construct the url for API access
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Hourly/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a1,b1,c1,d1,e1,f1 = set_args_list[5],set_args_list[0],set_args_list[1],set_args_list[2],set_args_list[3],set_args_list[4]
        access_link1 = url.format(a1,b1,c1,d1,e1,f1)
        response1 = requests.get(access_link1)
        data1 = response1.text
        data_df1 = pd.read_json(data1)
        return data_df1
        
    else:
        
        #construct the url for API access
        record_1 = set_args_list[1].replace(" ","%20")
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Hourly/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a2,b2,c2,d2,e2,f2 = set_args_list[5],set_args_list[0],record_1,set_args_list[2],set_args_list[3],set_args_list[4]
        access_link2 = url.format(a2,b2,c2,d2,e2,f2)
        response2 = requests.get(access_link2)
        data2 = response2.text
        data_df2 = pd.read_json(data2)
        return data_df2


## Data collection

### Get the list of all properties

In [ ]:
url = "https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List"
list_of_prpt_df = get_list_of_properties(url)
print(f"shape of property list dataframe is:{list_of_prpt_df.shape}")
list_of_prpt_df.head()

### Electricity consumption for 2022 (hourly, daily and monthly for 1000 Hakaniemen kauppahalli)

## Hourly

In [ ]:
elect_data_2022_hr = get_hourly_consumption(record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Electricity",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [ ]:
print(f"hourly data shape:{elect_data_2022_hr.shape}")
elect_data_2022_hr.head()

#### Decsriptive statistics of hourly electricity data

In [ ]:
elect_data_2022_hr['value'].describe()

## Daily

In [ ]:
elect_data_2022_daily = get_daily_consumption(record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Electricity",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [ ]:
print(f"hourly data shape:{elect_data_2022_daily.shape}")
elect_data_2022_daily.head()

## Monthly

In [ ]:
elect_data_2022_monthly = get_monthly_consumption(record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Electricity",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [ ]:
print(f"hourly data shape:{elect_data_2022_monthly.shape}")
elect_data_2022_monthly.head()

In [ ]:
list_of_prpt_df.info()

In [ ]:
list_of_prpt_df.isna().sum()

In [ ]:
null_property_name = list_of_prpt_df[list_of_prpt_df['propertyName'].isna()]
null_property_name

In [ ]:
null_property_code = list_of_prpt_df[list_of_prpt_df['propertyCode'].isna()]
null_property_code

**Observation on list of properties:**
    
    1. New property 1 and 2 is duplicated 
    2. 2352 Lpk oulunkylän lisätilat and 2316 Lpk Kumpupilvi has no property code
    3. There are 4 missing data in Property_name and 6 in Property_code
    
**Fix:**
    Missing data in property name can be resolved since it is the same as location_name
 

#### Fill missing property_name

In [ ]:
list_of_prpt_df.loc[600:, 'propertyName'] = list_of_prpt_df['locationName']

In [ ]:
list_of_prpt_df.isna().sum()

In [ ]:
null_property_name = list_of_prpt_df[list_of_prpt_df['propertyCode'].isna()]
null_property_name

In [ ]:
list_of_prpt_df['propertyName'].nunique() 

## Get electricity usage of "1000 Hakaniemen kauppahalli" for 2019

In [ ]:
# get the daily electricity usage for this property for 2019
response = requests.get(f"https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1000%20Hakaniemen%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31")
print(response.status_code)
hakaniemen_kaupa_1000_data = response.text
hakaniemen_kaupa_1000_df = pd.read_json(hakaniemen_kaupa_1000_data)
print(f"hakaniemen_kaupa_1000_df shape:{hakaniemen_kaupa_1000_df.shape}")
hakaniemen_kaupa_1000_df.head()

### Data type inspection

In [ ]:
# summary statistics
hakaniemen_kaupa_1000_df .info()

## Central and dispersion measure

In [ ]:
hakaniemen_kaupa_1000_df.describe()

### Distribution of the electricity usage

In [ ]:
# Plot the distribution of the propery electricity usage
sns.histplot(data=hakaniemen_kaupa_1000_df, x=hakaniemen_kaupa_1000_df['value'], kde=True)
plt.show()

### Monthly Consumption

In [ ]:
# get the daily electricity usage for this property for 2019
response = requests.get(f"https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Monthly/ListByProperty?Record=LocationName&SearchString=1000%20Hakaniemen%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31")
print(response.status_code)
hakaniemen_kaupa_monthly_data = response.text
hakaniemen_kaupa_monthly_df = pd.read_json(hakaniemen_kaupa_monthly_data)
print(f"hakaniemen_kaupa_monthly_df shape:{hakaniemen_kaupa_monthly_df.shape}")
hakaniemen_kaupa_monthly_df.head()

In [ ]:
hakaniemen_kaupa_monthly_df['month'] = pd.to_datetime()

In [ ]:
hakaniemen_kaupa_monthly_df ['timestamp'] = pd.to_datetime(hakaniemen_kaupa_monthly_df['timestamp'])
hakaniemen_kaupa_monthly_df['month'] = hakaniemen_kaupa_monthly_df ['timestamp'].dt.strftime('%b')
hakaniemen_kaupa_monthly_df.head()

In [ ]:
# Plot the line diagram for the monthly electricity consumption
sns.lineplot(x=hakaniemen_kaupa_monthly_df['month'], y=hakaniemen_kaupa_monthly_df['value'], data=hakaniemen_kaupa_monthly_df)
plt.show()

### District cooling Energy data

In [ ]:
def get_dailyly_consumption(
    record=None,
    search_string=None,
    reporting_group=None,
    start_time=None,
    end_time=None,
    Normalization=True,
    version=None
):
    # get a dictionary of all possible arguement
    all_args = locals()
    
    #fiter out the set arguements
    set_args = {k: v for k,v in all_args.items() if v is not None and k !='Normalization'}# and k != 'Normalization'
    
    # get a list of all values from the dict "set_args"
    set_args_list = list(set_args.values())
    
    if 'PropertyCode' in list(set_args.values()):
        #construct the url for API access
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Daily/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a1,b1,c1,d1,e1,f1 = set_args_list[5],set_args_list[0],set_args_list[1],set_args_list[2],set_args_list[3],set_args_list[4]
        access_link1 = url.format(a1,b1,c1,d1,e1,f1)
        response1 = requests.get(access_link1)
        data1 = response1.text
        data_df1 = pd.read_json(data1)
        return data_df1
        
    else:
        
        #construct the url for API access
        record_1 = set_args_list[1].replace(" ","%20")
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Daily/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a2,b2,c2,d2,e2,f2 = set_args_list[5],set_args_list[0],record_1,set_args_list[2],set_args_list[3],set_args_list[4]
        access_link2 = url.format(a2,b2,c2,d2,e2,f2)
        response2 = requests.get(access_link2)
        data2 = response2.text
        data_df2 = pd.read_json(data2)
        return data_df2

In [ ]:
def get_monthly_consumption(
    record=None,
    search_string=None,
    reporting_group=None,
    start_time=None,
    end_time=None,
    Normalization=True,
    version=None
):
    # get a dictionary of all possible arguement
    all_args = locals()
    
    #fiter out the set arguements
    set_args = {k: v for k,v in all_args.items() if v is not None and k !='Normalization'}# and k != 'Normalization'
    
    # get a list of all values from the dict "set_args"
    set_args_list = list(set_args.values())
    
    if 'PropertyCode' in list(set_args.values()):
        #construct the url for API access
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Monthly/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a1,b1,c1,d1,e1,f1 = set_args_list[5],set_args_list[0],set_args_list[1],set_args_list[2],set_args_list[3],set_args_list[4]
        access_link1 = url.format(a1,b1,c1,d1,e1,f1)
        response1 = requests.get(access_link1)
        data1 = response1.text
        data_df1 = pd.read_json(data1)
        return data_df1
        
    else:
        
        #construct the url for API access
        record_1 = set_args_list[1].replace(" ","%20")
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Monthly/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a2,b2,c2,d2,e2,f2 = set_args_list[5],set_args_list[0],record_1,set_args_list[2],set_args_list[3],set_args_list[4]
        access_link2 = url.format(a2,b2,c2,d2,e2,f2)
        response2 = requests.get(access_link2)
        data2 = response2.text
        data_df2 = pd.read_json(data2)
        return data_df2

In [ ]:
def get_hourly_consumption(
    record=None,
    search_string=None,
    reporting_group=None,
    start_time=None,
    end_time=None,
    Normalization=True,
    version=None
):
    # get a dictionary of all possible arguement
    all_args = locals()
    
    #fiter out the set arguements
    set_args = {k: v for k,v in all_args.items() if v is not None and k !='Normalization'}# and k != 'Normalization'
    
    # get a list of all values from the dict "set_args"
    set_args_list = list(set_args.values())
    
    if 'PropertyCode' in list(set_args.values()):
        #construct the url for API access
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Hourly/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a1,b1,c1,d1,e1,f1 = set_args_list[5],set_args_list[0],set_args_list[1],set_args_list[2],set_args_list[3],set_args_list[4]
        access_link1 = url.format(a1,b1,c1,d1,e1,f1)
        response1 = requests.get(access_link1)
        data1 = response1.text
        data_df1 = pd.read_json(data1)
        return data_df1
        
    else:
        
        #construct the url for API access
        record_1 = set_args_list[1].replace(" ","%20")
        url = 'https://helsinki-openapi.nuuka.cloud/api/v{}/EnergyData/Hourly/ListByProperty?Record={}&SearchString={}&ReportingGroup={}&StartTime={}&EndTime={}'
        a2,b2,c2,d2,e2,f2 = set_args_list[5],set_args_list[0],record_1,set_args_list[2],set_args_list[3],set_args_list[4]
        access_link2 = url.format(a2,b2,c2,d2,e2,f2)
        response2 = requests.get(access_link2)
        data2 = response2.text
        data_df2 = pd.read_json(data2)
        return data_df2

In [ ]:
were = get_hourly_consumption(record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Electricity",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [ ]:
print(f"dataframe shape:{were.shape}")
were.head()

In [ ]:
def get_list_of_properties(api_url):
    property_list = requests.get(api_url)
    if property_list.status_code !=200:
        print(f"request error:{property_list.status_code}")
    else:
        property_list_data = property_list.text
        propt_list_df = pd.read_json(property_list_data)
        return propt_list_df      

In [ ]:
text.replace("-"and" ","%")
if 'PropertyCode' in set_args.keys()


In [ ]:

list_2 = ['lizz', 'bens', 'cols']
if 'lizz' in list_2:
    print('yes')


In [ ]:
import os
import json
import shutil
import tarfile
import requests
import pandas as pd
from tqdm import tqdm
from io import BytesIO
from datetime import datetime
#from dotenv import load_dotenv, find_dotenv

In [ ]:
def get_data_request_as_json(
        id=None,
        mmsi=None,
        longitude=None,
        latitude=None,
        sog=None,
        cog=None,
        navstat=None,
        rot=None,
        posacc=None,
        raim=None,
        heading=None,
        timestamp=None,
        timestampexternal=None,
        metadata=False
):
    """ Convert field constraints into a json data field that the data platform API understands """

    # Get a dictionary of all possible arguments
    all_args = locals()
    # Filter out the arguments that are set
    set_args = {k: v for k, v in all_args.items() if v is not None and k != 'metadata'}
    
    return set_args

    # load up project environment variables to get the secret API key
    #dotenv_path = find_dotenv()
    #load_dotenv(dotenv_path)

    # Default template
   # data = {
    #    'meta': {'auth': os.environ.get("DATA_PLATFORM_API_KEY")},
    #    'constraints': []

In [ ]:
get_data_request_as_json()